In [ ]:
(in-package "ACL2")

In [ ]:
(include-book "centaur/fty/top" :dir :system)
(include-book "std/util/define" :dir :system)

In [ ]:
(fty::deftagsum chat-role
  (:system ())      ; System prompt / instructions
  (:user ())        ; User input
  (:assistant ())   ; Model response
  (:tool ()))       ; Tool result (for future MCP integration)

In [ ]:
;; Helper functions for creating roles
(defmacro role-system () '(chat-role-system))
(defmacro role-user () '(chat-role-user))
(defmacro role-assistant () '(chat-role-assistant))
(defmacro role-tool () '(chat-role-tool))

In [ ]:
(fty::defprod chat-message
  ((role chat-role-p "The role of the message sender")
   (content stringp "The text content of the message" :default ""))
  :layout :list)

In [ ]:
;; Convenient constructors
(define make-system-message ((content stringp))
  :returns (msg chat-message-p)
  (make-chat-message :role (role-system) :content content))

In [ ]:
(define make-user-message ((content stringp))
  :returns (msg chat-message-p)
  (make-chat-message :role (role-user) :content content))

In [ ]:
(define make-assistant-message ((content stringp))
  :returns (msg chat-message-p)
  (make-chat-message :role (role-assistant) :content content))

In [ ]:
(define make-tool-message ((content stringp))
  :returns (msg chat-message-p)
  (make-chat-message :role (role-tool) :content content))

In [ ]:
(fty::deflist chat-message-list
  :elt-type chat-message-p
  :true-listp t)

In [ ]:
;; Utility: Check if conversation has a system message
(define has-system-message-p ((messages chat-message-list-p))
  :returns (has booleanp)
  (if (endp messages)
      nil
    (let ((role (chat-message->role (car messages))))
      (or (chat-role-case role :system t :otherwise nil)
          (has-system-message-p (cdr messages))))))

In [ ]:
;; Utility: Get the last assistant message content (if any)
(define last-assistant-content ((messages chat-message-list-p))
  :returns (content stringp)
  (if (endp messages)
      ""
    (let* ((msg (car (last messages)))
           (role (chat-message->role msg)))
      (if (chat-role-case role :assistant t :otherwise nil)
          (chat-message->content msg)
        ""))))

In [ ]:
(define chat-role-to-string ((role chat-role-p))
  :returns (s stringp)
  (chat-role-case role
    :system "system"
    :user "user"
    :assistant "assistant"
    :tool "tool"))

In [ ]:
(define string-to-chat-role ((s stringp))
  :returns (role chat-role-p)
  (cond ((equal s "system") (role-system))
        ((equal s "user") (role-user))
        ((equal s "assistant") (role-assistant))
        ((equal s "tool") (role-tool))
        ;; Default to user for unknown roles
        (t (role-user))))

In [ ]:
(defthm chat-role-to-string-returns-valid-string
  (implies (chat-role-p role)
           (member-equal (chat-role-to-string role)
                        '("system" "user" "assistant" "tool")))
  :hints (("Goal" :in-theory (enable chat-role-to-string))))

In [ ]:
(defthm string-to-chat-role-roundtrip
  (implies (and (stringp s)
                (member-equal s '("system" "user" "assistant" "tool")))
           (equal (chat-role-to-string (string-to-chat-role s)) s))
  :hints (("Goal" :in-theory (enable chat-role-to-string string-to-chat-role))))

In [ ]:
(fty::deftagsum model-type
  (:llm ())         ; Language model for chat/completion
  (:vlm ())         ; Vision-language model
  (:embeddings ())) ; Embedding model

In [ ]:
(define model-type-to-string ((mt model-type-p))
  :returns (s stringp)
  (model-type-case mt
    :llm "llm"
    :vlm "vlm"
    :embeddings "embeddings"))

In [ ]:
(define string-to-model-type ((s stringp))
  :returns (mt model-type-p)
  (cond ((equal s "llm") (model-type-llm))
        ((equal s "vlm") (model-type-vlm))
        ((equal s "embeddings") (model-type-embeddings))
        ;; Default to llm for unknown
        (t (model-type-llm))))

In [ ]:
(fty::deftagsum model-state
  (:loaded ())
  (:not-loaded ()))

In [ ]:
(define model-state-to-string ((ms model-state-p))
  :returns (s stringp)
  (model-state-case ms
    :loaded "loaded"
    :not-loaded "not-loaded"))

In [ ]:
(define string-to-model-state ((s stringp))
  :returns (ms model-state-p)
  (if (equal s "loaded")
      (model-state-loaded)
    (model-state-not-loaded)))

In [ ]:
(fty::defprod model-info
  ((id stringp "Model identifier" :default "")
   (type model-type-p "Type: llm, vlm, or embeddings")
   (load-state model-state-p "Whether model is loaded")
   (publisher stringp "Model publisher" :default "")
   (arch stringp "Model architecture" :default "")
   (quantization stringp "Quantization level" :default "")
   (max-context-length natp "Maximum context length" :default 0)
   (loaded-context-length natp "Loaded context length (0 if not loaded)" :default 0))
  :layout :list)

In [ ]:
;; Predicate helpers
(define model-loaded-p ((m model-info-p))
  :returns (loaded booleanp)
  (let ((st (model-info->load-state m)))
    (model-state-case st
      :loaded t
      :not-loaded nil)))

In [ ]:
(define model-is-llm-p ((m model-info-p))
  :returns (is-llm booleanp)
  (let ((tp (model-info->type m)))
    (model-type-case tp
      :llm t
      :otherwise nil)))

In [ ]:
(define model-is-completions-p ((m model-info-p))
  :returns (is-completions booleanp)
  "True for models that can do chat/text completions (llm or vlm)"
  (let ((tp (model-info->type m)))
    (model-type-case tp
      :llm t
      :vlm t
      :embeddings nil)))

In [ ]:
(fty::deflist model-info-list
  :elt-type model-info-p
  :true-listp t)

In [ ]:
;; Filter helpers - use simpler approach without return type proofs
;; (ACL2 has trouble with recursive filter return types on FTY lists)
(defun filter-loaded-models (models)
  (declare (xargs :guard (model-info-list-p models)))
  (cond ((endp models) nil)
        ((model-loaded-p (model-info-fix (car models)))
         (cons (model-info-fix (car models)) 
               (filter-loaded-models (cdr models))))
        (t (filter-loaded-models (cdr models)))))

In [ ]:
(defun filter-completions-models (models)
  (declare (xargs :guard (model-info-list-p models)))
  (cond ((endp models) nil)
        ((model-is-completions-p (model-info-fix (car models)))
         (cons (model-info-fix (car models)) 
               (filter-completions-models (cdr models))))
        (t (filter-completions-models (cdr models)))))

In [ ]:
(defun filter-loaded-completions-models (models)
  (declare (xargs :guard (model-info-list-p models)))
  (filter-loaded-models (filter-completions-models models)))

In [ ]:
(fty::deftagsum llm-provider
  (:local ())       ; Local LM Studio instance
  (:openai ())      ; OpenAI API (gpt-4, gpt-3.5-turbo, etc.)
  (:anthropic ())   ; Anthropic API (claude-3, etc.) - future
  (:azure ())       ; Azure OpenAI - future
  (:custom ()))     ; Custom OpenAI-compatible endpoint

In [ ]:
(define llm-provider-to-string ((p llm-provider-p))
  :returns (s stringp)
  (llm-provider-case p
    :local "local"
    :openai "openai"
    :anthropic "anthropic"
    :azure "azure"
    :custom "custom"))

In [ ]:
(define string-to-llm-provider ((s stringp))
  :returns (p llm-provider-p)
  (cond ((equal s "local") (llm-provider-local))
        ((equal s "openai") (llm-provider-openai))
        ((equal s "anthropic") (llm-provider-anthropic))
        ((equal s "azure") (llm-provider-azure))
        ((equal s "custom") (llm-provider-custom))
        ;; Default to local for unknown
        (t (llm-provider-local))))

In [ ]:
(fty::defprod llm-provider-config
  ((provider llm-provider-p "Provider type")
   (endpoint stringp "API endpoint URL" :default "")
   (api-key stringp "API key (empty for local)" :default "")
   (model stringp "Default model to use" :default "")
   (org-id stringp "Organization ID (optional)" :default ""))
  :layout :list)

In [ ]:
;; Predefined provider configurations
(defmacro make-local-config (&key (model '""))
  `(make-llm-provider-config
    :provider (llm-provider-local)
    :endpoint "http://host.docker.internal:1234/v1/chat/completions"
    :api-key ""
    :model ,model
    :org-id ""))

In [ ]:
(defmacro make-openai-config (&key api-key (model '"gpt-4o-mini") (org-id '""))
  `(make-llm-provider-config
    :provider (llm-provider-openai)
    :endpoint "https://api.openai.com/v1/chat/completions"
    :api-key ,api-key
    :model ,model
    :org-id ,org-id))

In [ ]:
(defmacro make-custom-config (&key endpoint api-key model)
  `(make-llm-provider-config
    :provider (llm-provider-custom)
    :endpoint ,endpoint
    :api-key ,api-key
    :model ,model
    :org-id ""))

In [ ]:
;; Helper to check if provider requires API key
(define provider-requires-api-key-p ((config llm-provider-config-p))
  :returns (requires booleanp)
  (let ((prov (llm-provider-config->provider config)))
    (llm-provider-case prov
      :local nil
      :openai t
      :anthropic t
      :azure t
      :custom t)))

In [ ]:
;; Helper to check if config is valid (has API key if needed)
(define llm-config-valid-p ((config llm-provider-config-p))
  :returns (valid booleanp)
  (and (not (equal "" (llm-provider-config->endpoint config)))
       (or (not (provider-requires-api-key-p config))
           (not (equal "" (llm-provider-config->api-key config))))))